### Parseo GPX a CSV

#### Función ls(ruta)
Función que lista todos los ficheros y directorios que ese encuentran contenidos en la `ruta` que se especifica por parámetro:

In [1]:
from os import listdir

def ls(ruta = '.'):
    return listdir(ruta)

#### Función gpx_to_csv(rutaFichero)
Función que convierte un fichero GPX del cual se pasa su `ruta` y el nombre del fichero CSV (`nameCSV`) al cuál se va a convertir:

Estructura GPX:

    <gpx creator="StravaGPX" xmlns:xsi="..." 
                             xsi:schemaLocation="..." 
                             version="1.1" xmlns="..." 
                             xmlns:gpxtpx="..." 
                             xmlns:gpxx="...">
        <metadata>
          <time>2018-10-04T18:20:45Z</time>
        </metadata>
        <trk>
          <name>Carrera de noche</name>
          <type>9</type>
          <trkseg>
             <trkpt lat="43.4761380" lon="-3.7989940">
                <ele>4.5</ele>
                <time>2018-10-04T18:20:45Z</time>
                <extensions>
                     <gpxtpx:TrackPointExtension>
                          <gpxtpx:hr>168</gpxtpx:hr>
                     </gpxtpx:TrackPointExtension>
                </extensions>
             </trkpt>
             ...
          </trkseg>
           ...
        </trk>
    </gpx>
    

In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import re
import os

def gpx_to_csv(ruta):
    
    #Capturo todos los ficheros .gpx de la ruta
    gpx_docs= list()
    for doc in ls(ruta):
        if re.search('.gpx$', doc):
            gpx_docs.append(doc)
            
    for gpx in gpx_docs:    
      
        rutaFichero= ruta+"/"+gpx
        
        tree= ET.parse(rutaFichero)
        root = tree.getroot()
    
        #Creo el dataframe vacío donde guardaré los datos del .gpx
        df = pd.DataFrame(columns=('sport','time', 'latitud', 'longitud', 
                               'elevacion', 'hr', 'cadencia'))
        
        time=None #fecha y hora
        lat=None  #latitud
        lon=None  #longitud
        ele=None  #elevación
        hr=None   #heart rate
        cad=None  #cadencia
        sport=None
    
        #Coloco a trk como raíz
        for child in root:
            if("trk" in child.tag):
                root_trk=child
    
        #Coloco a trkseg como raíz
        for child in root_trk:
            if("type" in child.tag):
                sport = child.text
            if("trkseg" in child.tag):
                root_trkseg=child
            
        #Ahora que trkseg es la raíz, los hijos seran los track points
        #por lo que inicializo las variables a null que me servirán para 
        #añadir una fila al dataframe por cada track point.
    
        for trkpt in root_trkseg:
            lat = trkpt.attrib['lat']#latitud del trkpt
            lon = trkpt.attrib['lon']#longitud del trkpt
        
            for child_trkpt in trkpt: #leo los campos del trkpt
            
                if("ele" in child_trkpt.tag): #Si es el campo elevación
                    ele = child_trkpt.text #guardo la elevación del trkpt
            
                if("time" in child_trkpt.tag): #Si es el campo time
                    time = child_trkpt.text #guardo la fecha y la hora del trkpt
                
                if("extensions" in child_trkpt.tag): #Si es el campo de extensiones
                
                    extensions = child_trkpt #hago las extensiones raiz y leo sus campos
                
                    for gpxtpx in extensions:
                        for child_gpxtpx in gpxtpx:
                        
                            if("hr" in child_gpxtpx.tag): #Si tiene la extension hr
                                hr = child_gpxtpx.text #guardo el hr
                        
                            if("cad" in child_gpxtpx.tag): #Si tiene la extension cadencia
                                cad = child_gpxtpx.text #guardo la cadencia
                            
            #Recorrido ya el trackpoint lo guardo en el dataframe
            df.loc[len(df)] = [sport,time,lat,lon,ele,hr,cad]
                        
    
        #Recorrido ya el archivo, paso los datos del dataframe a un csv
        nuevo = gpx.replace(".gpx", ".csv")
        rutaNuevo = ruta+"/csv/"
        try:
            os.stat(rutaNuevo)
        except:
            os.mkdir(rutaNuevo)
        df.to_csv(rutaNuevo+nuevo, index = False)

### Parseo TCX a CSV

#### Función tcx_to_csv(rutaFichero)
Función que convierte un fichero TCX del cual se pasa su `ruta` y el nombre del fichero CSV (`nameCSV`) al cuál se va a convertir:

Esctructura TCX:

    <?xml version="1.0" encoding="UTF-8"?>
    <TrainingCenterDatabase
      xsi:schemaLocation="..." xmlns:ns5="..." xmlns:ns3="..." xmlns:ns2="..." xmlns="..." xmlns:xsi="..." xmlns:ns4="...">
       <Activities>
           <Activity Sport="Running">
              <Id>2016-01-03T08:18:23.000Z</Id>
              <Lap StartTime="2016-01-03T08:18:23.000Z">
                <TotalTimeSeconds>497.272</TotalTimeSeconds>
                <DistanceMeters>1000.0</DistanceMeters>
                <MaximumSpeed>2.921000003814697</MaximumSpeed>
                <Calories>110</Calories>
                <AverageHeartRateBpm>
                  <Value>160</Value>
                </AverageHeartRateBpm>
                <MaximumHeartRateBpm>
                  <Value>172</Value>
                </MaximumHeartRateBpm>
                <Intensity>Active</Intensity>
                <TriggerMethod>Manual</TriggerMethod>
                <Track>
                  <Trackpoint>
                    <Time>2016-01-03T08:18:23.000Z</Time>
                    <Position>
                      <LatitudeDegrees>43.39021844789386</LatitudeDegrees>
                      <LongitudeDegrees>-3.786661634221673</LongitudeDegrees>
                    </Position>
                    <AltitudeMeters>113.5999984741211</AltitudeMeters>
                    <DistanceMeters>0.75</DistanceMeters>
                    <HeartRateBpm>
                      <Value>90</Value>
                    </HeartRateBpm>
                    <Extensions>
                      <TPX xmlns="...">
                        <RunCadence>42</RunCadence>
                      </TPX>
                    </Extensions>
                  </Trackpoint>
                  ...
                </Track>
                <Extensions>
                  <LX xmlns="http://www.garmin.com/xmlschemas/ActivityExtension/v2">
                    <MaxRunCadence>92</MaxRunCadence>
                  </LX>
                  <LX xmlns="http://www.garmin.com/xmlschemas/ActivityExtension/v2">
                    <AvgRunCadence>78</AvgRunCadence>
                  </LX>
                  <LX xmlns="http://www.garmin.com/xmlschemas/ActivityExtension/v2">
                    <AvgSpeed>2.010999917984009</AvgSpeed>
                  </LX>
                  <LX xmlns="http://www.garmin.com/xmlschemas/ActivityExtension/v2">
                    <Steps>1298</Steps>
                  </LX>
                </Extensions>
              </Lap>
              ...
           </Activity> </Activities> </TrainingCenterDatabase>            

In [16]:
import xml.etree.ElementTree as ET
import pandas as pd
import re

def tcx_to_csv(ruta):
    
    #Capturo todos los ficheros .gpx de la ruta
    tcx_docs= list()
    for doc in ls(ruta):
        if re.search('.tcx$', doc):
            tcx_docs.append(doc)
    
    
    for tcx in tcx_docs:
        
        rutaFichero= ruta+"/"+tcx
        
        #Elimino el espacio inicial que añáde strava
        with open(rutaFichero, 'r') as file:
            data=file.read()

        data = data.lstrip()
        corregido = open(rutaFichero,"w")
        corregido.write(data)
        corregido.close()


        #Datos que se van a recoger del TCX
        time = None
        lat = None
        lon = None
        alt = None
        speed = None
        distance_trkpt = None
        hr_trkpt = None
        cad = None
        sport = None
        startTime = None
        totalTime = None
        distance = None
        calories = None
        steps = None
        maxSpeed = None
        avgSpeed = None
        maxHR = None
        avgHR = None
        maxCad = None
        avgCad = None
        

        #Creo los dataframes vacíos donde guardaré los datos del .tcx
        df_laps = pd.DataFrame(columns=('Sport','StartTime', 'TotalTime', 'Distance','Calories', 'Steps', 
                                        'MaxSpeed','AvgSpeed', 'MaxHR', 'AvgHR', 'MaxCadence', 'AvgCadence'))

        df_trkpts_lap = pd.DataFrame(columns=('id_lap', 'Time', 'Latitude', 'Longitude', 'Altitude', 'Speed', 
                                              'Distance', 'HR', 'Cadence'))

        #Parseo el documento
        tree= ET.parse(rutaFichero)
        root = tree.getroot()

        for child in root:
            if("Activities" in child.tag):
                act_root=child

        for activity in act_root:
            sport = activity.attrib["Sport"] #Tipo de actividad
            act_root=activity

        for child in act_root:
            if("Lap" in child.tag):
                lap_root= child
                startTime= child.attrib['StartTime']
                for child1 in lap_root:
                    if("TotalTimeSeconds" in child1.tag):
                        totalTime= child1.text
                    if("DistanceMeters" in child1.tag):
                        distance = child1.text
                    if("MaximumSpeed" in child1.tag):
                        maxSpeed = child1.text
                    if("Calories" in child1.tag):
                        calories = child1.text
                    if("AverageHeartRateBpm" in child1.tag):
                        for value in child1:
                            avgHR = value.text
                    if("MaximumHeartRateBpm" in child1.tag):
                        for value in child1:
                            maxHR = value.text
                    if("Extension" in child1.tag):
                        ext_lap_root = child1
                        for e in ext_lap_root:
                            for lx in e:
                                if("MaxRunCadence" in lx.tag):
                                    maxCad = lx.text
                                if("AvgRunCadence" in lx.tag):
                                    avgCad = lx.text
                                if("AvgSpeed" in lx.tag):
                                    avgSpeed = lx.text
                                if("Steps" in lx.tag):
                                    steps = lx.text
                                                        
                    if("Track" in child1.tag): #Track
                        trk_root = child1
                        for trkpt in trk_root: #Trackpoints
                            trkpt_root = trkpt
                    
                            for child_2 in trkpt_root: #Trackpoint childs
                                if("Time" in child_2.tag):
                                    time = child_2.text
                                if("AltitudeMeters" in child_2.tag):
                                    alt = child_2.text
                                if("DistanceMeters" in child_2.tag):
                                    distance_trkpt = child_2.text
                                if("HeartRateBpm" in child_2.tag):
                                    #print(child_2.tag)
                                    for value3 in child_2:
                                        #print(value3.tag, value3.text)
                                        hr_trkpt = value3.text
                                        #print(hr_trkpt)
                                if("Position" in child_2.tag):
                                    position_root = child_2
                                    for coord in position_root:
                                        if("Latitude" in coord.tag):
                                            lat = coord.text
                                        if("Longitude" in coord.tag):
                                            lon = coord.text
                                if("Cadence" in child_2.tag):
                                    cad = child_2.text
                                if("Extension" in child_2.tag):
                                    ext_trkpt_root = child_2
                                    for ext in ext_trkpt_root:
                                        for child_3 in ext:
                                            if("Cadence" in child_3.tag):
                                                cad = child_3.text
                                            if("Speed" in child_3.tag):
                                                speed = child_3.text
                            df_trkpts_lap.loc[len(df_trkpts_lap)] = [len(df_laps), time, lat, lon, alt,speed, distance_trkpt, hr_trkpt, cad]
        
                df_laps.loc[len(df_laps)] = [sport, startTime, totalTime, distance, calories, steps, maxSpeed, avgSpeed,
                                         maxHR, avgHR, maxCad, avgCad]
        
        #Recorrido ya el archivo, paso los datos del dataframe a un csv
        nuevaExt = tcx.replace(".tcx", ".csv")
        nuevoLaps = "laps_"+nuevaExt
        nuevoTrkpts= "trkpts_"+nuevaExt
        
        rutaNuevos = ruta + "/csv/"
        
        try:
            os.stat(rutaNuevos)
        except:
            os.mkdir(rutaNuevos)
        
        df_laps.to_csv(rutaNuevos+nuevoLaps, index = False)
        df_trkpts_lap.to_csv(rutaNuevos+nuevoTrkpts, index= False)

In [35]:
tcx_to_csv("./tcx")

### Conversión automática de los archivos

In [18]:
for i in range(3,8):
    
    rutaGeneral = "C:/Users/elsac/Documents/Master/TFM/Datos/activities_"+ str(i)
    directoriosRuta=ls(rutaGeneral)
    
    for directorio in directoriosRuta:
        if(directorio == "gpx"):
            rutaGpx = rutaGeneral + "/gpx"
            gpx_to_csv(rutaGpx)
        if(directorio == "tcx"):
            rutaTcx = rutaGeneral + "/tcx"
            tcx_to_csv(rutaTcx)
        